In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains  # 用于鼠标悬停
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

import matplotlib.pyplot as plt
# 如果需要 inline 显示
%matplotlib inline


In [2]:

# 启动 Edge 浏览器
driver = webdriver.Edge()

# 访问天津二手房首页
url = 'https://zu.fang.com/'
driver.get(url)

# 可视化效果时稍微等待下页面加载
sleep(2)


In [3]:
# 找到悬停的元素（s4Box）
hover_element = driver.find_element(By.CSS_SELECTOR, "div.s4Box")

# 创建动作对象
actions = ActionChains(driver)
# 将鼠标移动到目标元素
actions.move_to_element(hover_element).perform()

# 等待悬停后下拉列表出现
sleep(1)

# 在下拉列表中找到“天津”并点击
# 注意：这里使用 XPATH 或者 CSS 选择器，根据实际情况来
tianjin_city = driver.find_element(By.XPATH, "//div[@class='city20141104nr' and @id='cityi010']/a[text()='天津']")
tianjin_city.click()

# 切换到天津页面需要等待
sleep(2)


In [4]:
# 找到左侧区域列表中，“南开”对应的链接（注意这里的链接文字要和真实页面保持一致）
nankai_link = driver.find_element(By.LINK_TEXT, "南开")
nankai_link.click()

# 等待页面加载
sleep(2)


In [5]:
# 找到“八里台”对应的链接
balitai_link = driver.find_element(By.LINK_TEXT, "八里台")
balitai_link.click()

# 等待页面加载
sleep(2)


In [6]:
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

house_data = []  # 用于汇总所有页面的房源数据

while True:
    # ------------------ 1. 找到房源列表并解析当前页数据 ------------------
    try:
        # 新页面的容器：<div class="houseList">
        shop_list_container = driver.find_element(By.CSS_SELECTOR, "div.houseList")
    except NoSuchElementException:
        print("未找到房源列表的主容器，请检查页面结构或是否跳到最后一页。")
        break

    # 找到每一条房源的 dl 标签（class="list hiddenMap rel"）
    dl_elements = shop_list_container.find_elements(By.CSS_SELECTOR, "dl.list.hiddenMap.rel")

    for dl in dl_elements:
        try:
            # 标题
            title = dl.find_element(By.CSS_SELECTOR, "p.title a").text.strip()

            # 基本信息（如：整租|2室2厅|113㎡|朝南）
            house_info = dl.find_element(By.CSS_SELECTOR, "p.font15.mt12.bold").text.strip()
            
            # 月租金（如：2650）
            rent_price = dl.find_element(By.CSS_SELECTOR, "div.moreInfo .price").text.strip()

            # 存储信息
            house_data.append({
                "标题": title,
                "基本信息": house_info,   # 示例："整租|2室2厅|113㎡|朝南"
                "月租金(元/月)": rent_price
            })
        except NoSuchElementException:
            # 有些 dl 可能是广告位或特殊格式，忽略
            continue

    # ------------------ 2. 找到 “下一页” 按钮并翻页 ------------------
    try:
        time.sleep(2)  # 可以根据实际页面情况调整等待时间
        next_page = driver.find_element(By.LINK_TEXT, "下一页")  # 如果按钮文字变了，需要修改这里
        next_page.click()
        time.sleep(3)  # 等待页面加载
    except NoSuchElementException:
        print("没有找到下一页按钮，爬取结束！")
        break

# 全部页面爬取结束后，将 house_data 转为 DataFrame
df = pd.DataFrame(house_data)
df


没有找到下一页按钮，爬取结束！


,标题,基本信息,月租金(元/月)
0,免佣出租科海里 2650元 2室2厅2卫精装修无中间,整租|2室2厅|113㎡|朝南,2650
1,大安翠微园 精装2室 中间楼层 家电家具齐全 诚意出,整租|2室1厅|147㎡|朝南,6300
2,欣苑公寓 全明精装两室 位置优格局好 带车位 生态宜,整租|2室2厅|93㎡|朝南北,3300
3,望园里 精装2室空房 可以办公与居住 诚意出租 随时,整租|2室1厅|62㎡|朝南,2500
4,科海里 精装修 1室 让您找到家的温馨,整租|1室1厅|42㎡|朝南,2100
...,...,...,...
63,水上公园街(来福里)两居室 全新家电,整租|2室1厅|58㎡|朝南,2900
64,居祥里 一楼 精装房 临水上公园 诚意出租,整租|1室1厅|30㎡|朝南,1700
65,房子豪华装修大两居室俯瞰天塔外景视觉嘎嘎滴包物业啊出,整租|2室1厅|137㎡|朝南,8000
66,"观园公寓 2元 1室1厅1卫 精装修,全家私电器出租",整租|1室1厅|42㎡|朝南,2100


In [7]:
import pandas as pd
import re

# 复制 df 以防止修改原数据
df_cleaned = df.copy()

# 1. 提取面积（㎡）
def extract_area(info):
    """
    从 '基本信息' 字符串中提取面积（㎡），如 "整租|2室2厅|113㎡|朝南" 提取 "113"
    """
    match = re.search(r"\|(\d+\.?\d*)㎡\|", info)  # 匹配 "|113㎡|"
    return float(match.group(1)) if match else None

df_cleaned["面积(㎡)"] = df_cleaned["基本信息"].apply(extract_area)

# 2. 转换月租金为数值类型
df_cleaned["月租金(元/月)"] = df_cleaned["月租金(元/月)"].astype(float)

# 3. 仅保留需要的两列
df_cleaned = df_cleaned[["面积(㎡)", "月租金(元/月)"]]

# 4. 保存为 CSV 文件
df_cleaned.to_csv("rental_data_cleaned.csv", index=False)

print("数据已成功保存到 rental_data_cleaned.csv")


数据已成功保存到 rental_data_cleaned.csv
